In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.cluster import KMeans
import scipy as sp
import pickle
import matplotlib.pyplot as plt
from multiprocessing import pool
np.random.seed(0)

In [2]:
# creating neighborhood definition
stencil_mnf = tg.create_stencil("von_neumann", 1, 1)
# setting the center to zero
stencil_mnf.set_index([0,0,0], 0)

In [3]:
# loading the lattice from csv
lattice_path = os.path.relpath('../Data/dynamic output/voxelized_envelope_cut.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

In [4]:
neighs = init_avail_lattice.find_neighbours(stencil_mnf)
max_dist = np.sum(init_avail_lattice.shape)
env_lat_flat = init_avail_lattice.flatten()
n_voxel = init_avail_lattice.shape[0]*init_avail_lattice.shape[1]*init_avail_lattice.shape[2]


In [ ]:
mnfm = []
for n in range(n_voxel):
    street_connection_lattice = init_avail_lattice == -1
    street_connection_lattice[np.unravel_index(n, avail_lattice.shape)] +=1
    mn_dist_lattice = 1 - street_connection_lattice
    mn_dist_lattice[mn_dist_lattice==1] = max_dist
    mn_dist_lattice_flat = mn_dist_lattice.flatten()
    for i in range(1, max_dist):
        # find the neighbours of the previous step
        next_step = neighs[mn_dist_lattice_flat == i - 1]
        # find the unique neighbours
        next_unq_step = np.unique(next_step.flatten())
        # check if the neighbours of the next step are inside the envelope
        validity_condition = env_lat_flat[next_unq_step]
        # select the valid neighbours
        next_valid_step = next_unq_step[validity_condition]
        # make a copy of the lattice to prevent overwriting in the memory
        mn_nex_dist_lattice_flat = np.copy(mn_dist_lattice_flat)
        # set the next step cells to the current distance
        mn_nex_dist_lattice_flat[next_valid_step] = i
        # find the minimum of the current distance and previous distances to avoid overwriting previous steps
        mn_dist_lattice_flat = np.minimum(mn_dist_lattice_flat, mn_nex_dist_lattice_flat)
        
        # check how many of the cells have not been traversed yet
        filled_check = mn_dist_lattice_flat * env_lat_flat == max_dist
        # if all the cells have been traversed, break the loop
        if filled_check.sum() == 0:
            print(i)
            break
    mnfm.append(mn_dist_lattice_flat)

mn_dist_lattice = mnfm[1809].reshape(mn_dist_lattice.shape)

In [ ]:
# set the lattice to be visualized
l = mn_dist_lattice * init_avail_lattice

# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = l.shape
# The bottom left corner of the data set
grid.origin = l.minbound
# These are the cell sizes along each axis
grid.spacing = l.unit

# Add the data values to the cell data
grid.point_arrays["Street Access"] = l.flatten(order="F")  # Flatten the Lattice
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6]) * 1.5
p.add_volume(grid, cmap="YlGn", opacity=opacity, shade=False, show_scalar_bar=True)

# plotting
p.show(use_ipyvtk=True)